<a href="https://colab.research.google.com/github/harsh74444/Springer-Books/blob/master/springer_book.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
pip install tabula-py

In [0]:
pip install tabulate

In [0]:
import pandas as pd
from tabula import read_pdf
from tabulate import tabulate
import requests
import urllib
from bs4 import BeautifulSoup

user = input('Please provide the path of pdf file shared: ')
user_input_last = input('Please provide path for saving the file: ')


while True:
  pdf = read_pdf(user, pages='all', pandas_options={'header': None})

  lst_1 = pdf[0]
  df_head = pd.DataFrame(lst_1)
  df_head = df_head.drop(labels=0)
  headers = df_head.iloc[0]

  df_head = df_head[1:]

  df_head.columns = headers
  headers = ['S.No.', 'Book Title', 'Author', 'Edition', 'OpenURL']
  df1 = pd.DataFrame(columns = headers)
  for i in range(1,len(pdf)):
    lst = pdf[i]
    df = pd.DataFrame(lst)
    df.columns = headers
    df1 = pd.concat([df1,df])
    i += 1

  df_final = pd.concat([df_head, df1])
  df_final = df_final[df_final['Book Title'].notna()]
  df_final = df_final.reset_index(drop=True)
  df_final.index.name = 'S.No.'
  df_final = df_final.drop(columns =['S.No.'])

  user_input = input('Enter a search keyword: ')


  lst_input =[]
  for i in range(len(df_final)):
    if user_input.lower() in df_final['Book Title'][i].lower():
      lst_input.append([i, df_final.iloc[i, 0]])
      headers_input = ['Serial No.','Book Title']
      df_input = pd.DataFrame(lst_input, columns=headers_input)
    i+=1
  pd.options.display.max_columns
  print(df_input)

  while True:
    serial_input = eval(input('Enter Serial No. of desired Book: '))
    if serial_input in df_input['Serial No.'].values:
      print('Retrieving.....')
      break
  link = [df_final.iloc[serial_input,3]]

  url = link[0]
  html_content = requests.get(url).text
  soup = BeautifulSoup(html_content, 'lxml')

  url_1 = []
  for link in soup.find_all("div", {"class": "cta-button-container__item"}):
    for a in soup.findAll("a", {"title": "Download this book in PDF format"}):
      in_list = "https://link.springer.com" + a.get('href')
      url_1.append(in_list)

  if len(url_1) <1:
    print('Requested file is not for free')
  else:
    user_input_file_name = input('Please give your file a unique name: ')
    print('Please wait..!! Your file is Downloading...')
    urllib.request.urlretrieve(url_1[0], user_input_last+'/'+user_input_file_name)
    print('Your file is Downloaded.')

    user_input_more_file = input('Do you want to search for any other book? (Yes/No)')

  if user_input_more_file.lower() == 'yes':
    continue
  else:
    print('Thankyou for using this code.')
    print('Happy Learning..!!')
    break


